### กลุ่ม Diabetes Prediction 
1. 6610402205 นายรักษิต รุ่งรัตนไชย หมู่ 1
2. 6610402132 นายบวรรัตน์ ตั้งนรารัชชกิจ หมู่ 1
3. 6610401985 นายไชยวัตน์ หนูวัฒนา หมู่ 1  

### ลิงค์ไปยังข้อมูลที่จะใช้ในระบบต้นแบบ

In [2]:
import pandas as pd
diabetes = pd.read_csv("diabetes_binary_health_indicators_BRFSS2015.csv")
diabetes.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


### ข้อมูลมีการ/น่าจะมีการสุ่มตัวอย่างมา (sampling) แบบใด เพราะเหตุใด มีปัญหาหรือไม่ อย่างไร แก้ไขได้อย่างไร
น่าจะมีการสุ่มตัวอย่างมาแบบ Nonprobability Sampling(Quota sampling) เนื่องจากข้อมูลของผู้ที่เป็นและไม่ได้เป็นเบาหวานมีความห่างกันมากจำนวนไม่สมดุลกันและการเก็บข้อมูลเก็บจากการโทรศัพท์สำรวจเฉพาะกลุ่มคนที่มีพฤติกรรมเสี่ยงและมีปัญหาสุขภาพเรื้อรัง ซึ่งปัญหาที่พบก็คือข้อมูลไม่มีความสมดุลจึงต้องมีการแก้ไขในขั้นตอนถัดไป

In [3]:
diabetes["Diabetes_binary"].value_counts()

Diabetes_binary
0.0    218334
1.0     35346
Name: count, dtype: int64

### ข้อมูลมีการ/น่าจะมีการกำหนดเฉลย (labeling) ด้วยวิธีใด เพราะเหตุใด มีปัญหาหรือไม่ อย่างไร แก้ไขได้อย่างไร
ใช้วิธีกำหนดเฉลยแบบ Hand Labels เนื่องจากใช้วิธีการเก็บข้อมูลจากการโทรศัพท์สอบถามผู้คนในประเทศอเมริกาและทุกข้อมูลมี Labels ระบุคำตอบครบถ้วนจึงไม่มีปัญหาแต่อย่างใด

In [4]:
print("จำนวนข้อมูล: " , diabetes.shape[0] , "จำนวนผลเฉลย:", diabetes["Diabetes_binary"].value_counts().sum())

จำนวนข้อมูล:  253680 จำนวนผลเฉลย: 253680


### ข้อมูลขาดความสมดุล (imbalance) หรือไม่ อย่างไร มากน้อยแค่ไหน มีปัญหาหรือไม่ อย่างไร แก้ไขได้อย่างไร
ข้อมูลขาดความสมดุลเพราะ ข้อมูลของผู้ที่เป็นเบาหวานกับข้อมูลของผู้ที่ไม่ได้เป็นเบาหวานมีความห่างกันค่อนข้างมาก

In [5]:
diabetes["Diabetes_binary"].value_counts() # 1 คือเป็นเบาหวาน 0 คือไม่เป็นเบาหวาน

Diabetes_binary
0.0    218334
1.0     35346
Name: count, dtype: int64

อาจทำให้เกิดปัญหาการลำเอียงในการทำนายได้เราจึงใช้วิธีการ Resampling ใหม่โดยไปใช้วิธี Probability-based Samplilng แบบ Stratified Random Sample ในขั้นตอนการแบ่ง test train เพื่อนำไปเทรนโมเดล (ได้ลองใช้การ undersampling แล้วแต่ผลการทำนายและการหา feature importance ไม่สมเหตุสมผลจึงเลือกที่จะไม่ใช้เพราะอาจทำให้ข้อมูลจำนวนมากหายไปและทำให้ผลการทำนายคลาดเคลื่อนไป)

### จำเป็นหรือสามารถสร้างข้อมูลเพิ่ม (augmentation) ได้หรือไม่ อย่างไร
ไม่จำเป็นต้องทำ augmentation เนื่องจากข้อมูลไม่ได้เป็นรูปภาพเป็นในลักษณะของ True False หรือ 0 1 จึงไม่มีความจำเป็นต้องทำ augmentation

### ประเด็นอื่น ๆ ที่เกี่ยวข้องกับข้อมูลของตัวเอง
เนื่องจากข้อมูลมีความ imbalance จึงลองที่จะทำ undersampling เพื่อตัดข้อมูลให้เท่ากันคาดว่าจะทำให้มีความลำเอียง(bias) น้อยลงแต่หลังจากที่ทดลองทดสอบด้วยโมเดล LogisticRegression พบว่าประสิทธิภาพในการทำนายลดลงและ feature importance ไม่สมเหตุสมผลอาจทำให้การทำนายไม่แม่นยำจึงเปลี่ยนมาใช้วิธีการ sampling แบบ Probability-based Samplilng(Stratified Random Sample) ในขั้นตอนการแบ่ง train test แทน ซึ่งประสิทธิภาพเป็นที่น่าพอใจ

### การมีส่วนร่วมของสมาชิกแต่ละคนในกลุ่ม
- นายรักษิต รุ่งรัตนไชย ทำหน้าที่เขียนอธิบายในหัวข้อต่างๆที่สรุปได้จากข้อมูล
- นายบวรรัตน์ ตั้งนรารัชชกิจ สืบเสาะที่มาของข้อมูลเพื่อนำมาวิเคราห์การทำ sampling
- นายไชยวัตน์ หนูวัฒนา ทำหน้าที่ทดลองโมเดลเพื่อแก้ปัญหา imbalance 

### การเปิดเผยการใช้เครื่องมือปัญญาประดิษฐ์ (ใช้อะไร ใช้เพื่ออะไร ใช้อย่างไร, prompt อย่างไร)
ไม่ได้มีการใช้ AI หรือเครื่องมือปัญญาประดิษฐ์ใดๆในงานนี้